<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/train_realfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

Cloning into 'realfill'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 313 (delta 66), reused 35 (delta 28), pack-reused 216 (from 1)
Receiving objects: 100% (313/313), 1.28 MiB | 3.34 MiB/s, done.
Resolving deltas: 100% (137/137), done.
/content/realfill


In [5]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  452M  100  452M    0     0  64.5M      0  0:00:07  0:00:07 --:--:-- 64.5M


In [6]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/jensen_images.zip -o jensen_images.zip
!unzip -q jensen_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 43.5M  100 43.5M    0     0  16.7M      0  0:00:02  0:00:02 --:--:-- 25.5M


In [7]:
%pwd
%ls -lh

total 497M
drwxr-xr-x 4 root root 4.0K Apr 21 16:03 data/
-rw-r--r-- 1 root root 3.4K Apr 21 16:03 infer.py
drwxr-xr-x 3 root root 4.0K Apr 22  2025 jensen_images/
-rw-r--r-- 1 root root  44M Apr 21 16:03 jensen_images.zip
-rw-r--r-- 1 root root 1.1K Apr 21 16:03 LICENSE
drwxr-xr-x 3 root root 4.0K Apr 21 16:03 __MACOSX/
-rw-r--r-- 1 root root  383 Apr 21 16:03 README.md
-rw-r--r-- 1 root root 5.9K Apr 21 16:03 README-Realfill.md
drwxr-xr-x 4 root root 4.0K May 30  2024 realfill_data_release_full/
-rw-r--r-- 1 root root 453M Apr 21 16:03 realfill_data_release_full.zip
-rw-r--r-- 1 root root  119 Apr 21 16:03 requirements.txt
-rw-r--r-- 1 root root 6.4K Apr 21 16:03 train_realfill.ipynb
-rw-r--r-- 1 root root  37K Apr 21 16:03 train_realfill.py


In [8]:
!uv pip install -U -r requirements.txt --no-progress

Using Python 3.11.12 environment at: /usr
Resolved 58 packages in 560ms
 Downloaded pillow
 Downloaded tensorboard
 Downloaded setuptools
 Downloaded nvidia-cuda-cupti-cu12
 Downloaded diffusers
 Downloaded nvidia-nvjitlink-cu12
 Downloaded nvidia-cuda-nvrtc-cu12
 Downloaded nvidia-curand-cu12
 Downloaded xformers
 Downloaded bitsandbytes
 Downloaded numpy
 Downloaded nvidia-cusolver-cu12
 Downloaded nvidia-cusparse-cu12
 Downloaded nvidia-cufft-cu12
 Downloaded nvidia-cublas-cu12
 Downloaded nvidia-cudnn-cu12
Prepared 25 packages in 21.69s
Uninstalled 22 packages in 215ms
Installed 25 packages in 58ms
 - absl-py==1.4.0
 + absl-py==2.2.2
 - accelerate==1.5.2
 + accelerate==1.6.0
 + bitsandbytes==0.45.5
 - diffusers==0.32.2
 + diffusers==0.33.1
 + ftfy==6.3.1
 - numpy==2.0.2
 + numpy==2.2.5
 - nvidia-cublas-cu12==12.5.3.2
 + nvidia-cublas-cu12==12.4.5.8
 - nvidia-cuda-cupti-cu12==12.5.82
 + nvidia-cuda-cupti-cu12==12.4.127
 - nvidia-cuda-nvrtc-cu12==12.5.82
 + nvidia-cuda-nvrtc-cu12==12

In [9]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [14]:
import os

# Toggle switch - set to True for Google Drive storage, False for local storage
USE_DRIVE_STORAGE = True
# Toggle switch - set to True for FP32 training, False for FP16 (mixed precision)
USE_FP32 = False # Set to True to use FP32 precision

# Base directory for Google Drive storage
DRIVE_BASE_DIR = "/content/drive/MyDrive/RealFill"

# Keep your original environment variable setup
os.environ["DATASET"] = "jensen_images" # realfill_data_release_full, jensen_images
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
os.environ["BENCHMARK"] = "Custom"  # RealBench, Qualitative, Custom
os.environ["DATASET_NUMBER"] = "35"

# Construct paths based on the storage toggle
if USE_DRIVE_STORAGE:
    # Use Google Drive for storage
    from google.colab import drive # Original import location
    drive.mount('/content/drive') # Original mount call
    base_output_prefix = f'{DRIVE_BASE_DIR}/{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}'
else:
    # Use local Colab storage (original paths)
    base_output_prefix = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}'

# --- Set output paths directly combining the conditional suffix ---
os.environ["OUTPUT_DIR"] = f'{base_output_prefix}-model{"-fp32" if USE_FP32 else ""}'
os.environ["OUTPUT_IMG_DIR"] = f'{base_output_prefix}-results{"-fp32" if USE_FP32 else ""}'

# The remaining variable paths stay the same (Original)
os.environ["TRAIN_DIR"] = f'{os.environ["DATASET"]}/{os.environ["BENCHMARK"]}/{os.environ["DATASET_NUMBER"]}'
os.environ["VAL_IMG"] = f'{os.environ["TRAIN_DIR"]}/target/target.png'
os.environ["VAL_MASK"] = f'{os.environ["TRAIN_DIR"]}/target/mask.png'

# Create the necessary directories (Original)
os.makedirs(os.environ["OUTPUT_DIR"], exist_ok=True)
os.makedirs(os.environ["OUTPUT_IMG_DIR"], exist_ok=True)

# --- Set the shell environment variable using os.environ ---
# This makes the variable available to the subshell created by !
os.environ['PRECISION_ARG'] = '--mixed_precision=fp16' if not USE_FP32 else ''

# --- Original Print Statements ---
print(f"Using {'Google Drive' if USE_DRIVE_STORAGE else 'local Colab'} storage")
print(f"Output directory: {os.environ['OUTPUT_DIR']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using Google Drive storage
Output directory: /content/drive/MyDrive/RealFill/Custom-35-model


In [ ]:
!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100 \
  --checkpointing_steps 100 \
  $PRECISION_ARG \
  --use_8bit_adam \
  --set_grads_to_none \
  --enable_xformers_memory_efficient_attention

2025-04-21 16:04:53.513208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745251493.532457    3332 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745251493.538298    3332 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-21 16:04:53.557552: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has n

In [ ]:
!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

In [ ]:
# Zip final inference results
!zip -r9j $OUTPUT_IMG_DIR.zip $OUTPUT_IMG_DIR
# Zip tensorboard logs
!zip -r9D $OUTPUT_DIR-tensorboard.zip $OUTPUT_DIR/logs
%ls